In [ ]:
import numpy as np
import pandas as pd
import torch 
from torchvision import datasets,transforms
import torch.nn.functional as F
from torch import optim,nn
from torchsummary import summary 
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv("train.csv")

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data.iloc[index]
               
        image = item[1:].values.astype(np.uint8).reshape((28, 28))
        label = item[0]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
transform_train = transforms.Compose([transforms.ToPILImage(),transforms.Pad(2),transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
transform_valid = transforms.Compose([transforms.ToPILImage(),transforms.Pad(2),transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

In [ ]:
valid_size = 0.15
indicies = len(dataset)
r = list(range(indicies))
np.random.shuffle(r)
split = int(np.floor(valid_size * indicies))
train_idx, valid_idx = r[split:],r[:split]

In [ ]:
train_data = Data(dataset, transform = transform_train)
valid_train = Data(dataset,transform = transform_valid)

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64 , sampler = train_sampler)
validloader = torch.utils.data.DataLoader(valid_train, batch_size = 64, sampler = valid_sampler)

data = iter(trainloader)
X, Y = data.next()
print(X.shape)
print(Y.shape)

In [ ]:
print(f"Length train: {len(train_idx)}")
print(f"Length valid: {len(valid_idx)}")

In [ ]:
'''fig, axis = plt.subplots(3, 10, figsize=(15, 10))
images, labels = next(iter(trainloader))

for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label = images[i], labels[i]

        ax.imshow(image.view(28, 28), cmap='binary') # add image
        ax.set(title = f"{label}") # add label
        '''

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()

        self.pool = nn.MaxPool2d(kernel_size = 2 ,stride = 2 )
        self.dropout = nn.Dropout(p = 0.4)

        self.L = nn.Conv2d( in_channels = 1, out_channels = 4, kernel_size=(3,3),padding= 1)
        self.L1 = nn.Conv2d(in_channels = 5, out_channels = 4, kernel_size=(3,3),padding = 1 )
        self.L2 = nn.Conv2d(in_channels = 9, out_channels = 4, kernel_size=(3,3),padding = 1)
        self.L3 = nn.Conv2d(in_channels = 13, out_channels = 4, kernel_size=(3,3),padding = 1 )
        self.L4 = nn.Conv2d(in_channels = 17, out_channels = 4, kernel_size=(3,3),padding = 1 )
       

        self.batchnorm_c1 = nn.BatchNorm2d(5)
        self.batchnorm_c2 = nn.BatchNorm2d(9)
        self.batchnorm_c3 = nn.BatchNorm2d(13)
        self.batchnorm_c4 = nn.BatchNorm2d(17)
        self.batchnorm_c5 = nn.BatchNorm2d(21)
       
        self.dropout1 = nn.Dropout(p = 0.4)

        self.conv1 = nn.Conv2d(in_channels = 21, out_channels = 32, kernel_size = (3,3),padding = 1,stride = 1 )
        self.batchnorm1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size= (3,3), stride = 1)
        self.batchnorm2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size= (3,3),padding =1,stride = 1)
        self.batchnorm3 = nn.BatchNorm2d(128)

        self.linear1 = nn.Linear(128*3*3,120)
        self.linear2 = nn.Linear(120,84)
        self.linear3 = nn.Linear(84,64)
        self.linear4 = nn.Linear(64,10)


    def forward(self,X):

        z0 = self.L(X)
        z0c = torch.cat((X,z0),dim=1)
        z0b = self.batchnorm_c1(z0c)
        a0 = F.relu(z0b)

        z1 = self.L1(a0)
        z1c = torch.cat((X,z0,z1),dim = 1)
        z1b = self.batchnorm_c2(z1c)
        a1 = F.relu(z1b)

        z2 = self.L2(a1)
        z2c = torch.cat((X,z0,z1,z2),dim = 1)
        z2b = self.batchnorm_c3(z2c)
        a2 = F.relu(z2b)

        z3 = self.L3(a2)
        z3c = torch.cat((X,z0,z1,z2,z3),dim = 1)
        z3b = self.batchnorm_c4(z3c)
        a3 = F.relu(z3b)

        z4 = self.L4(a3)
        z4c = torch.cat((X,z0,z1,z2,z3,z4),dim = 1)
        z4b = F.dropout(self.batchnorm_c5(z4c))
        a4 = F.relu(z4b)

       


        z11 = self.conv1(a4)
        a11 = self.pool(self.dropout(self.batchnorm1(F.relu(z11))))

        z22 = self.conv2(a11)
        a22 = self.pool(self.dropout(self.batchnorm2(F.relu(z22))))

        z33 = self.conv3(a22)
        a33 = self.pool(self.dropout(self.batchnorm3(F.relu(z33))))


        a33 = a33.view(a33.shape[0],-1)

        a44 = self.dropout(F.relu(self.linear1(a33)))
        a55 = self.dropout(F.relu(self.linear2(a44))) 
        a66 = self.dropout(F.relu(self.linear3(a55)))
        a77 = F.log_softmax(self.linear4(a66), dim = 1)


        return a77

In [ ]:
model = Model()

model.cuda()

In [ ]:
summary(model,input_size=(1,32,32))

In [ ]:
epoch = 100
alpha = 0.001
criterion = nn.NLLLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = alpha)

In [ ]:
train_loss_data = []
valid_loss_data = []
train_acc_data = []
valid_acc_data = []

for e in range(epoch):
    train_loss = 0
    valid_loss = 0
    train_acc  = 0
    valid_acc  = 0
    
    model.train()
    
    for images,labels in trainloader:
        
        image = images.cuda()
        label = labels.cuda()

        logps = model(image)
        
        
        optimizer.zero_grad()
        
        loss = criterion(logps,label.squeeze())
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item() * image.size(0)
        
        ps = torch.exp(logps)
        top_p,top_class = ps.topk(1,dim=1)
        T_equals = top_class == label.view(*top_class.shape)
        
        train_acc += torch.mean(T_equals.type(torch.FloatTensor))
        
    
    
    model.eval()
    
    for images,labels in validloader:
        image = images.cuda()
        label = labels.cuda()
        
        logps = model(image)
        loss = criterion(logps,label)
        valid_loss += loss.item() * images.size(0)
        
        ps = torch.exp(logps)
        top_p,top_class = ps.topk(1,dim=1)

        equals = top_class == label.view(*top_class.shape)
        valid_acc += torch.mean(equals.type(torch.FloatTensor))
        
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
    train_acc = train_acc/len(trainloader)
    valid_acc = valid_acc/len(validloader)
    
    train_loss_data.append(train_loss)
    valid_loss_data.append(valid_loss)
    train_acc_data.append(train_acc)
    valid_acc_data.append(valid_acc)
    
    
    print("Epoch : {} Training Loss : {:.6f} Validation Loss : {:.6f} Traning Accuracy : {:.3f} Validation Accuracy : {:.3f}".format(e+1,train_loss,valid_loss,train_acc,valid_acc))        

In [ ]:
torch.save(model.state_dict(),'model.pt')

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
class DatasetSubmissionMNIST(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data.iloc[index].values.astype(np.uint8).reshape((28, 28))

        
        if self.transform is not None:
            image = self.transform(image)
            
        return image

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Pad(2),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

submissionset = DatasetSubmissionMNIST(test_data, transform=transform)
submissionloader = torch.utils.data.DataLoader(submissionset, batch_size=64, shuffle=False)

In [ ]:
data = iter(submissionloader)
X = data.next()
print(X.shape)

In [ ]:
submission = [['ImageId', 'Label']]

with torch.no_grad():
    model.eval()
    image_id = 1

    for images in submissionloader:
        
        images = images.cuda()
        log_ps = model(images)
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        
        for prediction in top_class:
            submission.append([image_id, prediction.item()])
            image_id += 1
            
print(len(submission) - 1)

In [ ]:
import csv

with open('submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(submission)
    
print('Submission Complete!')

In [ ]:
sub = pd.read_csv("submission.csv")